<div align="center"; span style="color:#336699"><b><h2>Minicurso pyForTraCC </h2></b></div>
<div align="center"; span style="color:#336699"><b><h3>1. Exemplo introdutório</h3></b></div>
<hr style="border:2px solid #0077b9;">
<br/>
<div style="text-align: center;font-size: 90%;">
   <sup>Autor: <a href="https://www.linkedin.com/in/helvecio-leal/"> Helvécio B. Leal Neto, <i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup><t>&nbsp;</t> 
   <br/><br/>
    National Institute for Space Research (INPE)
    <br/>
    Avenida dos Astronautas, 1758, Jardim da Granja, São José dos Campos, SP 12227-010, Brazil
    <br/><br/>
    Contact: <a href="mailto:fortracc.project@inpe.br">fortracc.project@inpe.br</a>
    <br/><br/>
    Last Update: Sep 6, 2025
</div>

<br/>

<div style="text-align: center; margin: 0 auto; width: 80ch;">
<b>Resumo.</b> Este notebook faz parte do minicurso de introdução ao <a href="https://github.com/fortracc/pyfortracc">pyfortracc</a> e apresenta um exemplo básico de rastreamento de dados sintéticos</a>.
</div>

<div align="center">

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/fortracc/pyfortracc/blob/main/examples/WORCAP-Minicourse/1_Basic_Tracking/1_Basic_Tracking.ipynb) 

</div>
<br/>

### Roteiro
 [1. Instalação](#install)<br>
 [2. Dados de Entrada](#input)<br>
 [3. Função de Leitura](#data)<br>
 [4. Parâmetros (Name_list)](#namelist)<br>
 [5. Rotina de Rastreamento](#track)<br>
 [6. Tabela de Rastreamento](#tracktable)<br>
 [7. Visualização de Rastreamento](#visualization)<br>


#### 1. **Instalação** <a class="anchor" id="install"></a>
Para instalar a biblioteca pyFortraCC, você pode usar o pip. Execute o seguinte comando

In [ ]:
# Comando para instalação da ultima versão estável do pyfortracc
!python -m pip install -qqq -U pyfortracc > /dev/null 2>&1 && echo "✅ pyfortracc instalado com sucesso!" || echo "❌ Erro na instalação"

Após a instalação caso tudo ocorra bem você verá a mensagem "✅ pyfortracc instalado com sucesso!". 

Em seguida, você pode importar a biblioteca e verificar a versão instalada.

In [ ]:
# Importa o pyfortracc e mostra a versão instalada
import pyfortracc
print('pyFortracc version', pyfortracc.__version__)

#### 2. **Dados de Entrada** <a class="anchor" id="input"></a>

Para este exemplo, usaremos dados sintéticos gerados por uma função interna do pyfortracc chamada `bubble_simulation()`. Esta função cria um conjunto de dados fictícios que simulam a movimentação de objetos ao longo do tempo e armazena esses dados como imagens no formato png no diretório especificado.

In [ ]:
# Gera arquivos de simulação de bolhas
pyfortracc.utilities.bubble_simulation(dir='input/')

Caso tudo ocorra bem, você poderá explorar o conteúdo das simulações geradas dentro do diretório especificado. Neste caso, o diretório é `.input/`. Você pode navegar até esse diretório para visualizar os arquivos de imagem gerados pela simulação. Cada arquivo de imagem representa um instante no tempo da simulação, mostrando a posição dos objetos simulados naquele momento específico. Com o comando abaixo, você pode listar os arquivos presentes no diretório `.input/` para verificar se as imagens foram criadas corretamente.

In [ ]:
# Lista os arquivos gerados no diretório de input
!ls -s input/

#### 3. **Função de Leitura** <a class="anchor" id="data"></a>
A função `read_data()` é responsável por ler os dados de entrada, que neste caso são imagens PNG. Ela utiliza a biblioteca PIL (Python Imaging Library) para abrir e converter as imagens em arrays binários.  
Essa função retorna um array numpy contendo os dados lidos de cada imagem.  
Para `pyfortracc`, cada imagem é tratada como um "frame" ou "time step" na sequência temporal dos dados, e a saída da função deve ser um array numpy 2D.

A célula abaixo demonstra como deve ser definida a função `read_data()` para ler imagens PNG.

In [ ]:
from PIL import Image
import numpy as np

# Define a função de leitura personalizada
def read_function(path):
    # Abre a imagem e converte para escala de cinza
    img = Image.open(path).convert("L")
    # Converte a imagem em um array NumPy
    arr = np.array(img)
    # Binariza a imagem: bolhas (1) e fundo (0)
    arr = np.where(arr < 250, 1, 0)
    return arr

Para verificar se a função `read_data()` está funcionando corretamente, você pode tentar abrir uma das imagens geradas pela simulação.  A célula abaixo tenta abrir a imagem `input/frame_00.png` <br>e exibir seu conteúdo como um array numpy. Se a função estiver correta, você verá a matriz de valores binários representando a imagem.

In [ ]:
# Testa a função de leitura
read_function('input/frame_00.png')

Caso tudo ocorra bem, você verá a matriz de valores binários com dimensões de (50, 50), indicando que a imagem foi lida corretamente.<br>
O pyfortracc também conta com uma função interna chamada `plot_animation()` que permite visualizar a sequência temporal dos dados lidos.<br>
A célula abaixo demonstra como usar essa função para criar uma animação dos frames lidos pela função `read_data()`.<br>
Como argumentos da função, você deve passar o caminho dos arquivos de entrada `input/*.png` e a função de leitura `read_function`.

In [ ]:
# Visualiza a animação dos frames de entrada
pyfortracc.plot_animation(path_files='input/*.png', 
                          read_function=read_function)

#### 4. **Parâmetros (Name_list)** <a class="anchor" id="namelist"></a>
O dicionário `name_list` é uma estrutura de dados que armazena os parâmetros necessários para o processo de rastreamento no pyfortracc. <br>
Cada chave do dicionário representa um parâmetro específico, e o valor associado a essa chave define o comportamento do rastreamento.<br> Você pode encontrar uma descrição detalhada de cada parâmetro na [documentação oficial do pyfortracc](https://fortracc.github.io/pyfortracc/).

Vamos entender cada um dos parâmetros definidos no `name_list`:

**`input_path` e `output_path`**: Definem os diretórios de entrada e saída dos dados, respectivamente.

**`thresholds`**: Lista de valores de limiar usados para segmentar os dados. Objetos com valores maiores ou iguais a este limiar serão considerados como clusters. No nosso exemplo, usamos `[1]`, indicando que qualquer pixel com valor ≥ 1 será considerado parte de um objeto.

**`min_cluster_size`**: Lista que define o tamanho mínimo (em pixels) para que um grupo de pixels conectados seja considerado um cluster válido. Clusters menores que este valor serão descartados. Usamos `[3]`, significando que clusters com menos de 3 pixels serão ignorados.

**`operator`**: Define o operador de comparação usado com os thresholds. Pode ser:
- `'>='`: maior ou igual (padrão)
- `'<='`: menor ou igual  
- `'=='`: igual

**`timestamp_pattern`**: Padrão para extrair informações de tempo dos nomes dos arquivos. No exemplo, `'frame_%M.png'` significa que o número após "frame_" representa minutos. Outros padrões possíveis:
- `'%Y%m%d_%H%M.png'`: para arquivos como "20230915_1430.png"
- `'data_%d_%H.png'`: para arquivos como "data_15_14.png"

**`delta_time`**: Intervalo de tempo entre frames consecutivos (em minutos). Este valor é crucial para calcular velocidades e trajetórias dos objetos.

In [ ]:
# Define os parâmetros de entrada para o rastreamento
name_list = {}
name_list['input_path'] = 'input/' # Caminho para os arquivos de entrada
name_list['output_path'] = 'output/' # Caminho para os arquivos de saída
name_list['thresholds'] = [1] # Lista de limiares de intensidade a serem usados no processo de segmentação
name_list['min_cluster_size'] = [3] # Lista que contém o tamanho mínimo dos clusters
name_list['operator'] = '>=' # '>= - <=' ou '=='
name_list['timestamp_pattern'] = 'frame_%M.png' # Padrão de nome de arquivo de timestamp
name_list['delta_time'] = 1 # em minutos

#### 5. **Rotina de Rastreamento** <a class="anchor" id="track"></a>

Agora que temos nossos dados preparados, função de leitura definida e parâmetros configurados, podemos executar a rotina principal de rastreamento do pyfortracc.

A função `pyfortracc.track()` é o coração do algoritmo e realiza as seguintes etapas:

1. **Feature Extraction**: Extrai características dos objetos detectados, como área, centroide, intensidade máxima, etc.
2. **Spatial Operations**: Realiza operações espaciais para identificar e conectar objetos entre frames.
3. **Linking**: Conecta objetos entre frames consecutivos baseado em proximidade e características
4. **Concat**: Concatena os resultados em uma tabela de rastreamento.

**Importante**: Esta operação pode levar alguns segundos dependendo do tamanho dos dados e parámetros definidos.

In [ ]:
# Executa o rastreamento
pyfortracc.track(name_list, read_function)

#### 6. **Tabela de Rastreamento** <a class="anchor" id="tracktable"></a>

Após executar o rastreamento, o pyfortracc gera diversos arquivos de saída contendo os resultados. Um dos principais outputs é a **tabela de rastreamento**, que contém informações detalhadas sobre cada objeto rastreado ao longo do tempo.

A tabela de rastreamento em multiplos arquivos Parquet (um para cada frame) contém colunas como:

* **`timestamp`** (datetime64[us]): Informação temporal do cluster.  
* **`uid`** (float64): Identificador único do cluster.  
* **`iuid`** (float64): Identificador único interno do cluster.  
* **`threshold_level`** (int64): Nível do limiar.  
* **`threshold`** (float64): Limiar específico.  
* **`status`** (object): Status da entidade (NEW, CON, SPL, MRG, SPL/MRG)  
* **`u_, v_`** (float64): Componentes do vetor.  
* **`inside_clusters`** (object): Número de clusters internos.  
* **`size`** (int64): Tamanho do cluster em pixels.  
* **`min, mean, max, std`** (float64): Estatísticas descritivas.  
* **`delta_time`** (timedelta64[us]): Variação temporal.  
* **`file`** (object): Nome do arquivo associado.  
* **`array_y, array_x`** (object): Coordenadas do array do cluster.  
* **`vector_field`** (object): Campo vetorial associado.  
* **`trajectory`** (object): Trajetória do cluster.  
* **`geometry`** (object): Representação geométrica da fronteira do cluster.  
* **`lifetime`** (int64): Vida útil do cluster em minutos.

Para carregar a tabela de rastreamento, você pode usar a biblioteca pandas para ler os arquivos Parquet gerados. A célula abaixo demonstra como fazer isso.

In [ ]:
# Importa as bibliotecas necessárias
import glob
import pandas as pd

# Carrega todos os arquivos de rastreamento no diretório de saída
tracking_files = sorted(glob.glob(name_list['output_path'] + '/track/trackingtable/*.parquet'))
tracking_table = pd.concat(pd.read_parquet(f) for f in tracking_files)

tracking_table.head(3)

Para selecionar clusters específicos, você pode usar os métodos de filtragem do pandas. Com isso, será possível analisar o comportamento de um cluster individual ao longo do tempo. A célula abaixo demonstra como selecionar clusters com `uid` igual a 1.

In [ ]:
# Seleciona clusters com uid igual a 1
selected_clusters = tracking_table[tracking_table['uid'] == 1]
selected_clusters.head(3)

Para explorar por exemplo as características de tamanho dos clusters ao longo do tempo, você pode plotar gráficos que mostrem a evolução dessas características. A célula abaixo demonstra como criar um gráfico simples da evolução do tamanho dos clusters com `uid` igual a 1 ao longo do tempo.

In [ ]:
# Plotar a evolução do tamanho dos clusters com uid igual a 1 ao longo do tempo
selected_clusters.plot(x='timestamp', y='size', marker='o', title='Evolução do Tamanho do Cluster com uid=1')

#### 7. **Visualização de Rastreamento** <a class="anchor" id="visualization"></a>

O pyfortracc também oferece funcionalidades para visualizar os resultados do rastreamento. Você pode visualizar o rastreamento para um frame específico ou criar uma animação mostrando a evolução dos clusters ao longo do tempo.

In [ ]:
# Visualiza o rastreamento para um frame específico no tempo '1900-01-01 00:19:00'
pyfortracc.plot(name_list=name_list, read_function=read_function, timestamp='1900-01-01 00:19:00')

Ou você pode criar uma animação mostrando a evolução dos clusters ao longo do tempo.

In [ ]:
# Cria uma animação mostrando a evolução dos clusters ao longo do tempo
pyfortracc.plot_animation(name_list= name_list, # Passa a lista de nomes
                          read_function=read_function, # Passa a função de leitura
                          start_timestamp= tracking_table['timestamp'].min().strftime('%Y-%m-%d %H:%M:%S'), # Data e hora de início no formato 'YYYY-MM-DD HH:MM:SS'
                          end_timestamp= tracking_table['timestamp'].max().strftime('%Y-%m-%d %H:%M:%S'), # Data e hora de fim no formato 'YYYY-MM-DD HH:MM:SS'
                          info_cols=['uid','lifetime', 'status'] # Colunas de informação a serem exibidas na animação
)

### 8. **Conclusão** <a class="anchor" id="conclusion"></a>

**Parabéns!** Você concluiu com sucesso o primeiro exemplo do pyfortracc. Neste tutorial, você aprendeu:

- Como instalar e configurar o pyfortracc
- Como preparar e ler dados de entrada
- Como definir parâmetros essenciais para o rastreamento
- Como executar a rotina de rastreamento
- Como interpretar a tabela de rastreamento gerada
- Como visualizar os resultados do rastreamento

### Recursos Adicionais

- [Documentação Oficial](https://pyfortracc.readthedocs.io/)
- [Repositório GitHub](https://github.com/fortracc/pyfortracc)
- [Suporte](mailto:fortracc.project@inpe.br)
- [Artigos Científicos](https://pyfortracc.readthedocs.io/BI/BI_PUBLICATIONS.html)


### Dicas para Casos Reais
Quando aplicar o pyfortracc aos seus próprios dados:

1. **Ajuste os thresholds** baseado nas características dos seus dados
2. **Configure corretamente** o `timestamp_pattern` e `delta_time`
3. **Defina `min_cluster_size`** apropriado para filtrar ruído
4. **Use parâmetros avançados** como `memory` e `vel_continuity` para melhorar o linking
5. **Monitore o uso de memória** em datasets grandes